# Prepare filelists for LJSpeech dataset


In [1]:
# See: https://github.com/espeak-ng/espeak-ng/blob/master/docs/languages.md
dir_data = "/home/temduck/vits2/datasets/custom_base"
config = "../config.yaml"
symlink = "F2"
n_val = 100
n_test = 500

In [2]:
! ls

filelists.ipynb


## Get hyperparameters from config file


In [3]:
import pandas as pd
from utils.hparams import get_hparams_from_file

hps = get_hparams_from_file(config)

## Read dataset

Here `normalized_text` contains numbers in the form of words.

**Note**: you may need to replace all `"|"` with `" | "` in the file `metadata.csv`.


In [18]:
data = pd.read_csv(
    "../F2.csv", 
    index_col=0
)
data["cleaned_text"] = None
data.head()

,file,text,normalized_text,cleaned_text
0,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_126...,ақтөбе облысының әкімі оңдасын оразалин өнеркә...,ақтөбе облысының әкімі оңдасын оразалин өнеркә...,None
1,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_725...,Түнде бұрқасын көтеріледі.\n,түнде бұрқасын көтеріледі.,None
2,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_246...,елбасы нұр-сұлтан назарбаев жолаушылар ұшағыны...,елбасы нұр-сұлтан назарбаев жолаушылар ұшағыны...,None
3,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_442...,олардың плей-офф кезеңіне шығуға мүмкіндігі зо...,олардың плей-офф кезеңіне шығуға мүмкіндігі зор.,None
4,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_968...,бұл оқу орындарында жыл сайын түлектерді даярл...,бұл оқу орындарында жыл сайын түлектерді даярл...,None


In [ ]:
data = pd.read_csv(
    f"{dir_data}/metadata_copy.csv",
    sep=r"|",
    header=None,
    names=["file","", "text", "normalized_text", "cleaned_text"],
    index_col=False,
    # converter to add .wav to file name
    converters={"file": lambda x: f"{symlink}/{x.strip()}.wav", "text": str.strip, "normalized_text": str.strip},
)
data.head()

## Text cleaners

It may take a while, so better to preprocess the text and save it to a file in advance.

**Note** `phonemize_text` takes the longest time.`


In [19]:
# Get index of tokenize_text
text_cleaners = hps.data.text_cleaners

token_idx = text_cleaners.index("tokenize_text")
token_cleaners = text_cleaners[token_idx:]
print(token_cleaners)


# Extract phonemize_text
def separate_text_cleaners(text_cleaners):
    final_list = []
    temp_list = []

    for cleaner in text_cleaners:
        if cleaner == "phonemize_text":
            if temp_list:
                final_list.append(temp_list)
            final_list.append([cleaner])
            temp_list = []
        else:
            temp_list.append(cleaner)

    if temp_list:
        final_list.append(temp_list)

    return final_list


text_cleaners = text_cleaners[:token_idx]
text_cleaners = separate_text_cleaners(text_cleaners)
print(text_cleaners)

['tokenize_text', 'add_bos_eos']
[['phonemize_text'], ['add_spaces']]


In [20]:
from text import tokenizer
from torchtext.vocab import Vocab

text_norm = data["normalized_text"].tolist()
for cleaners in text_cleaners:
    print(f"Cleaning with {cleaners} ...")
    if cleaners[0] == "phonemize_text":
        text_norm = tokenizer(text_norm, Vocab, cleaners, language=hps.data.language)
    else:
        for idx, text in enumerate(text_norm):
            temp = tokenizer(text, Vocab, cleaners, language=hps.data.language)
            text_norm[idx] = temp

data = data.assign(cleaned_text=text_norm)
data.head()

/home/temduck/anaconda3/envs/vits2/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/temduck/anaconda3/envs/vits2/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


Cleaning with ['phonemize_text'] ...
Cleaning with ['add_spaces'] ...


,file,text,normalized_text,cleaned_text
0,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_126...,ақтөбе облысының әкімі оңдасын оразалин өнеркә...,ақтөбе облысының әкімі оңдасын оразалин өнеркә...,ɑ q t ɵ b ˈe <space> ˈo b ɫ ə s ə n ə ŋ <space...
1,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_725...,Түнде бұрқасын көтеріледі.\n,түнде бұрқасын көтеріледі.,t u n d ˈe <space> b ʊ ɾ q ˈɑ s ə n <space> k ...
2,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_246...,елбасы нұр-сұлтан назарбаев жолаушылар ұшағыны...,елбасы нұр-сұлтан назарбаев жолаушылар ұшағыны...,e ɫ b ˈɑ s ə <space> n ˈʊ ɾ s ʊ ɫ t ˈɑ n <spac...
3,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_442...,олардың плей-офф кезеңіне шығуға мүмкіндігі зо...,олардың плей-офф кезеңіне шығуға мүмкіндігі зор.,o ɫ ˈɑ ɾ d ə ŋ <space> p l ˈe j ˈo f f <space>...
4,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_968...,бұл оқу орындарында жыл сайын түлектерді даярл...,бұл оқу орындарында жыл сайын түлектерді даярл...,b ˈʊ ɫ <space> ˈo q w <space> ˈo ɾ ə n d ɑ ɾ ə...


## Generate and save vocabulary


In [21]:
from torchtext.vocab import build_vocab_from_iterator
from utils.task import load_vocab, save_vocab
from text.symbols import special_symbols, UNK_ID
from typing import List


def yield_tokens(cleaned_text: List[str]):
    for text in cleaned_text:
        yield text.split()


text_norm = data["cleaned_text"].tolist()
vocab = build_vocab_from_iterator(yield_tokens(text_norm), specials=special_symbols)
vocab.set_default_index(UNK_ID)

vocab_file = f"../vocab.txt"
save_vocab(vocab, vocab_file)

vocab = load_vocab(vocab_file)
print(f"Size of vocabulary: {len(vocab)}")
print(vocab.get_itos())

Size of vocabulary: 59
['<pad>', '<unk>', '<bos>', '<eos>', '<space>', '<laugh>', 'ə', 'ɑ', 'n', 't', 'ˈɑ', 'ɾ', 'e', 's', 'ɫ', 'd', 'q', 'j', 'm', 'k', 'ɪ', 'ˈɪ', 'ˈe', 'b', 'ʒ', 'w', 'z', 'ʀ', 'ŋ', 'p', 'ʃ', 'ˈo', 'o', 'ɡ', 'l', '.', ',', 'æ', 'ɵ', 'u', 'ˈʊ', 'v', 'ʊ', 'a', 'x', 'ˈu', 'ˈɵ', 'f', 'ʔ', 'ˈæ', 'ˈa', 'tʃ', ':', ';', '?', '!', 'h', '(en)', '(kk)']


## Token cleaners


In [22]:
from text import detokenizer

text_norm = data["cleaned_text"].tolist()
for idx, text in enumerate(text_norm):
    temp = tokenizer(text, vocab, token_cleaners, language=hps.data.language)
    assert UNK_ID not in temp, f"Found unknown symbol:\n{text}\n{detokenizer(temp)}"
    text_norm[idx] = temp

text_norm = ["\t".join(map(str, text)) for text in text_norm]
data = data.assign(tokens=text_norm)
data.head()

,file,text,normalized_text,cleaned_text,tokens
0,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_126...,ақтөбе облысының әкімі оңдасын оразалин өнеркә...,ақтөбе облысының әкімі оңдасын оразалин өнеркә...,ɑ q t ɵ b ˈe <space> ˈo b ɫ ə s ə n ə ŋ <space...,2\t7\t16\t9\t38\t23\t22\t4\t31\t23\t14\t6\t13\...
1,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_725...,Түнде бұрқасын көтеріледі.\n,түнде бұрқасын көтеріледі.,t u n d ˈe <space> b ʊ ɾ q ˈɑ s ə n <space> k ...,2\t9\t39\t8\t15\t22\t4\t23\t42\t11\t16\t10\t13...
2,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_246...,елбасы нұр-сұлтан назарбаев жолаушылар ұшағыны...,елбасы нұр-сұлтан назарбаев жолаушылар ұшағыны...,e ɫ b ˈɑ s ə <space> n ˈʊ ɾ s ʊ ɫ t ˈɑ n <spac...,2\t12\t14\t23\t10\t13\t6\t4\t8\t40\t11\t13\t42...
3,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_442...,олардың плей-офф кезеңіне шығуға мүмкіндігі зо...,олардың плей-офф кезеңіне шығуға мүмкіндігі зор.,o ɫ ˈɑ ɾ d ə ŋ <space> p l ˈe j ˈo f f <space>...,2\t32\t14\t10\t11\t15\t6\t28\t4\t29\t34\t22\t1...
4,/Datasets/ISSAI_KazakhTTS2/F2/Audio/inform_968...,бұл оқу орындарында жыл сайын түлектерді даярл...,бұл оқу орындарында жыл сайын түлектерді даярл...,b ˈʊ ɫ <space> ˈo q w <space> ˈo ɾ ə n d ɑ ɾ ə...,2\t23\t40\t14\t4\t31\t16\t25\t4\t31\t11\t6\t8\...


## Save train, val, test filelists


In [23]:
data = data[["file", "tokens"]]
data = data.sample(frac=1).reset_index(drop=True)

data_train = data.iloc[n_val + n_test:]
data_val = data.iloc[:n_val]
data_test = data.iloc[n_val: n_val + n_test]

data_train.to_csv("../filelists/train.txt", sep="|", index=False, header=False)
data_val.to_csv("../filelists/val.txt", sep="|", index=False, header=False)
data_test.to_csv("../filelists/test.txt", sep="|", index=False, header=False)

In [ ]:
import pandas as pd
from pathlib import Path 

test = pd.read_csv("filelists/test.txt", sep="|", names=["file", "tokens"])
test.file = test.file.apply(lambda x: x.replace("/Datasets/ISSAI_KazakhTTS2/", ""))
train = pd.read_csv("filelists/train.txt", sep="|", names=["file", "tokens"])
train.file = train.file.apply(lambda x: x.replace("/Datasets/ISSAI_KazakhTTS2/", ""))
val = pd.read_csv("filelists/val.txt", sep="|", names=["file", "tokens"])
val.file = val.file.apply(lambda x: x.replace("/Datasets/ISSAI_KazakhTTS2/", ""))

train.to_csv("filelists/train.txt", sep="|", index=False, header=False)
val.to_csv("filelists/val.txt", sep="|", index=False, header=False)
test.to_csv("filelists/test.txt", sep="|", index=False, header=False)